This jupyter notebook aims to retrieve data from http://www.taiwanlottery.com.tw/lotto/Lotto649/history.aspx.

In [1]:
import requests
import pandas as pd
from lxml import etree
from lxml import html
from time import sleep

In [2]:
def getData(year, month):
    view_state = "/wEPDwUKMjA1NDMxNDc2OA9kFgICAQ9kFgICAw9kFhACAQ8QZGQWAQICZAIDDxAPFgIeB0NoZWNrZWRoZGRkZAIFDw8WBB4EVGV4dGUeB0VuYWJsZWRoZGQCBw8QDxYCHwBnZGRkZAIJDxAPFgIfAmdkZGRkAgsPEA8WAh8CZ2RkZGQCDw8PFgIfAWVkZAIRDzwrAAkBAA8WBB4IRGF0YUtleXMWAB4LXyFJdGVtQ291bnQCCmQWFGYPZBZQAgEPDxYCHwEFCTEwNTAwMDA2M2RkAgMPDxYCHwEFCTEwNS8wNy8xNWRkAgUPDxYCHwEFCTEwNS8xMC8xNWRkAgcPDxYCHwEFCjg5LDc5MSwwMDBkZAIJDw8WAh8BBQsxNzUsNDQyLDI1OGRkAgsPDxYCHwEFAjI5ZGQCDQ8PFgIfAQUCMjRkZAIPDw8WAh8BBQIyNWRkAhEPDxYCHwEFAjM1ZGQCEw8PFgIfAQUCMzNkZAIVDw8WAh8BBQIzOGRkAhcPDxYCHwEFAjQ5ZGQCGQ8PFgIfAQUCMjRkZAIbDw8WAh8BBQIyNWRkAh0PDxYCHwEFAjI5ZGQCHw8PFgIfAQUCMzNkZAIhDw8WAh8BBQIzNWRkAiMPDxYCHwEFAjM4ZGQCJQ8PFgIfAQUCNDlkZAInDw8WAh8BBQEwZGQCKQ8PFgIfAQUBMWRkAisPDxYCHwEFAjIzZGQCLQ8PFgIfAQUCNjNkZAIvDw8WAh8BBQUxLDU3M2RkAjEPDxYCHwEFBTEsOTYwZGQCMw8PFgIfAQUGMTksMzgxZGQCNQ8PFgIfAQUGMjgsNTA4ZGQCNw8PFgIfAQUBMGRkAjkPDxYCHwEFCTMsMjcyLDkzMGRkAjsPDxYCHwEFBjczLDc0NmRkAj0PDxYCHwEFBjE3LDk0OGRkAj8PDxYCHwEFBTEsOTc2ZGQCQQ8PFgIfAQUFMSwwMDBkZAJDDw8WAh8BBQM0MDBkZAJFDw8WAh8BBQM0MDBkZAJHDw8WAh8BBQsxNDUsMTE3LDE0M2RkAkkPDxYCHwEFATBkZAJLDw8WAh8BBQEwZGQCTQ8PFgIfAQUBMGRkAk8PDxYCHwEFATBkZAIBD2QWUAIBDw8WAh8BBQkxMDUwMDAwNjJkZAIDDw8WAh8BBQkxMDUvMDcvMTJkZAIFDw8WAh8BBQkxMDUvMTAvMTJkZAIHDw8WAh8BBQo4NCw2OTYsMzUwZGQCCQ8PFgIfAQULMTUxLDg3OSw0ODFkZAILDw8WAh8BBQI0MmRkAg0PDxYCHwEFAjMxZGQCDw8PFgIfAQUCNDZkZAIRDw8WAh8BBQI0M2RkAhMPDxYCHwEFAjI1ZGQCFQ8PFgIfAQUCMzRkZAIXDw8WAh8BBQIwNWRkAhkPDxYCHwEFAjI1ZGQCGw8PFgIfAQUCMzFkZAIdDw8WAh8BBQIzNGRkAh8PDxYCHwEFAjQyZGQCIQ8PFgIfAQUCNDNkZAIjDw8WAh8BBQI0NmRkAiUPDxYCHwEFAjA1ZGQCJw8PFgIfAQUBMGRkAikPDxYCHwEFATBkZAIrDw8WAh8BBQIxMmRkAi0PDxYCHwEFAjUwZGQCLw8PFgIfAQUFMSwyODhkZAIxDw8WAh8BBQUxLDk2MmRkAjMPDxYCHwEFBjIxLDI4M2RkAjUPDxYCHwEFBjI0LDU3MWRkAjcPDxYCHwEFATBkZAI5Dw8WAh8BBQEwZGQCOw8PFgIfAQUHMTMxLDM5NmRkAj0PDxYCHwEFBjIxLDAyM2RkAj8PDxYCHwEFBTIsMjQ0ZGQCQQ8PFgIfAQUFMSwwMDBkZAJDDw8WAh8BBQM0MDBkZAJFDw8WAh8BBQM0MDBkZAJHDw8WAh8BBQsxMjQsNDgwLDQ0NWRkAkkPDxYCHwEFCTEsNTc2LDc2M2RkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAgIPZBZQAgEPDxYCHwEFCTEwNTAwMDA2MWRkAgMPDxYCHwEFCTEwNS8wNy8wOGRkAgUPDxYCHwEFCTEwNS8xMC8wOGRkAgcPDxYCHwEFCjY5LDYwOCw2NTBkZAIJDw8WAh8BBQsxMzIsODQ4LDg4OWRkAgsPDxYCHwEFAjEzZGQCDQ8PFgIfAQUCMTBkZAIPDw8WAh8BBQIwOWRkAhEPDxYCHwEFAjE5ZGQCEw8PFgIfAQUCMzVkZAIVDw8WAh8BBQIyM2RkAhcPDxYCHwEFAjM4ZGQCGQ8PFgIfAQUCMDlkZAIbDw8WAh8BBQIxMGRkAh0PDxYCHwEFAjEzZGQCHw8PFgIfAQUCMTlkZAIhDw8WAh8BBQIyM2RkAiMPDxYCHwEFAjM1ZGQCJQ8PFgIfAQUCMzhkZAInDw8WAh8BBQEwZGQCKQ8PFgIfAQUBMWRkAisPDxYCHwEFAjI1ZGQCLQ8PFgIfAQUCODFkZAIvDw8WAh8BBQUxLDU1NmRkAjEPDxYCHwEFBTEsODEwZGQCMw8PFgIfAQUGMTgsMTM2ZGQCNQ8PFgIfAQUGMjYsMDkzZGQCNw8PFgIfAQUBMGRkAjkPDxYCHwEFCTQsMTA2LDMzNmRkAjsPDxYCHwEFBjQ1LDA3OWRkAj0PDxYCHwEFBTksMjc1ZGQCPw8PFgIfAQUFMSwzMjdkZAJBDw8WAh8BBQUxLDAwMGRkAkMPDxYCHwEFAzQwMGRkAkUPDxYCHwEFAzQwMGRkAkcPDxYCHwEFCzEwNSwyOTYsNDg5ZGQCSQ8PFgIfAQUBMGRkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAgMPZBZQAgEPDxYCHwEFCTEwNTAwMDA2MGRkAgMPDxYCHwEFCTEwNS8wNy8wNWRkAgUPDxYCHwEFCTEwNS8xMC8wNWRkAgcPDxYCHwEFCjgyLDYzOSw1NTBkZAIJDw8WAh8BBQsxMjEsNDA3LDMxNWRkAgsPDxYCHwEFAjA5ZGQCDQ8PFgIfAQUCMTdkZAIPDw8WAh8BBQI0MGRkAhEPDxYCHwEFAjA2ZGQCEw8PFgIfAQUCMTVkZAIVDw8WAh8BBQI0N2RkAhcPDxYCHwEFAjI0ZGQCGQ8PFgIfAQUCMDZkZAIbDw8WAh8BBQIwOWRkAh0PDxYCHwEFAjE1ZGQCHw8PFgIfAQUCMTdkZAIhDw8WAh8BBQI0MGRkAiMPDxYCHwEFAjQ3ZGQCJQ8PFgIfAQUCMjRkZAInDw8WAh8BBQEwZGQCKQ8PFgIfAQUBMGRkAisPDxYCHwEFAjIzZGQCLQ8PFgIfAQUCNzJkZAIvDw8WAh8BBQUxLDczNWRkAjEPDxYCHwEFBTIsMDI1ZGQCMw8PFgIfAQUGMTksNTg3ZGQCNQ8PFgIfAQUGMzAsMDkyZGQCNw8PFgIfAQUBMGRkAjkPDxYCHwEFATBkZAI7Dw8WAh8BBQY2MSw0NDhkZAI9Dw8WAh8BBQYxMywwODZkZAI/Dw8WAh8BBQUxLDQ5M2RkAkEPDxYCHwEFBTEsMDAwZGQCQw8PFgIfAQUDNDAwZGQCRQ8PFgIfAQUDNDAwZGQCRw8PFgIfAQUKOTEsNTg0LDc4NWRkAkkPDxYCHwEFCTIsOTc5LDM0N2RkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAgQPZBZQAgEPDxYCHwEFCTEwNTAwMDA1OWRkAgMPDxYCHwEFCTEwNS8wNy8wMWRkAgUPDxYCHwEFCTEwNS8xMC8wMWRkAgcPDxYCHwEFCjgyLDQ4NiwyNTBkZAIJDw8WAh8BBQsxMDAsNzAzLDQ5N2RkAgsPDxYCHwEFAjQxZGQCDQ8PFgIfAQUCMzRkZAIPDw8WAh8BBQIyMGRkAhEPDxYCHwEFAjQzZGQCEw8PFgIfAQUCMjRkZAIVDw8WAh8BBQIwOGRkAhcPDxYCHwEFAjMyZGQCGQ8PFgIfAQUCMDhkZAIbDw8WAh8BBQIyMGRkAh0PDxYCHwEFAjI0ZGQCHw8PFgIfAQUCMzRkZAIhDw8WAh8BBQI0MWRkAiMPDxYCHwEFAjQzZGQCJQ8PFgIfAQUCMzJkZAInDw8WAh8BBQEwZGQCKQ8PFgIfAQUBMGRkAisPDxYCHwEFAjI4ZGQCLQ8PFgIfAQUCNjBkZAIvDw8WAh8BBQUxLDIyNmRkAjEPDxYCHwEFBTEsODM4ZGQCMw8PFgIfAQUGMTgsODY5ZGQCNQ8PFgIfAQUGMjQsNzAzZGQCNw8PFgIfAQUBMGRkAjkPDxYCHwEFATBkZAI7Dw8WAh8BBQY1NSw5MjlkZAI9Dw8WAh8BBQYxNyw0MDBkZAI/Dw8WAh8BBQUyLDM0MWRkAkEPDxYCHwEFBTEsMDAwZGQCQw8PFgIfAQUDNDAwZGQCRQ8PFgIfAQUDNDAwZGQCRw8PFgIfAQUKNzQsMzg5LDUyNWRkAkkPDxYCHwEFCTEsNTY2LDAzOGRkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAgUPZBZQAgEPDxYCHwEFCTEwNTAwMDA1OGRkAgMPDxYCHwEFCTEwNS8wNi8yOGRkAgUPDxYCHwEFCTEwNS8wOS8yOGRkAgcPDxYCHwEFCjgwLDI4NiwwMDBkZAIJDw8WAh8BBQo4Myw2NzcsMDg4ZGQCCw8PFgIfAQUCMjVkZAINDw8WAh8BBQI0NmRkAg8PDxYCHwEFAjQ1ZGQCEQ8PFgIfAQUCMDhkZAITDw8WAh8BBQIzOWRkAhUPDxYCHwEFAjA3ZGQCFw8PFgIfAQUCMzhkZAIZDw8WAh8BBQIwN2RkAhsPDxYCHwEFAjA4ZGQCHQ8PFgIfAQUCMjVkZAIfDw8WAh8BBQIzOWRkAiEPDxYCHwEFAjQ1ZGQCIw8PFgIfAQUCNDZkZAIlDw8WAh8BBQIzOGRkAicPDxYCHwEFATBkZAIpDw8WAh8BBQExZGQCKw8PFgIfAQUCMjhkZAItDw8WAh8BBQI2MWRkAi8PDxYCHwEFBTEsMzc5ZGQCMQ8PFgIfAQUFMSw5MzJkZAIzDw8WAh8BBQYyMCw0OTBkZAI1Dw8WAh8BBQYyNSw2OTdkZAI3Dw8WAh8BBQEwZGQCOQ8PFgIfAQUJMiw5NDYsNjIzZGQCOw8PFgIfAQUGNTAsODkzZGQCPQ8PFgIfAQUGMTUsNTc0ZGQCPw8PFgIfAQUFMSw4OTRkZAJBDw8WAh8BBQUxLDAwMGRkAkMPDxYCHwEFAzQwMGRkAkUPDxYCHwEFAzQwMGRkAkcPDxYCHwEFCjU1LDMzNiwwNjBkZAJJDw8WAh8BBQEwZGQCSw8PFgIfAQUBMGRkAk0PDxYCHwEFATBkZAJPDw8WAh8BBQEwZGQCBg9kFlACAQ8PFgIfAQUJMTA1MDAwMDU3ZGQCAw8PFgIfAQUJMTA1LzA2LzI0ZGQCBQ8PFgIfAQUJMTA1LzA5LzI0ZGQCBw8PFgIfAQUKODMsNzExLDQ1MGRkAgkPDxYCHwEFCjY1LDUyNiw3NjhkZAILDw8WAh8BBQIxN2RkAg0PDxYCHwEFAjE0ZGQCDw8PFgIfAQUCMzlkZAIRDw8WAh8BBQI0MWRkAhMPDxYCHwEFAjI3ZGQCFQ8PFgIfAQUCMjhkZAIXDw8WAh8BBQIxOGRkAhkPDxYCHwEFAjE0ZGQCGw8PFgIfAQUCMTdkZAIdDw8WAh8BBQIyN2RkAh8PDxYCHwEFAjI4ZGQCIQ8PFgIfAQUCMzlkZAIjDw8WAh8BBQI0MWRkAiUPDxYCHwEFAjE4ZGQCJw8PFgIfAQUBMGRkAikPDxYCHwEFATBkZAIrDw8WAh8BBQIzMmRkAi0PDxYCHwEFAjU3ZGQCLw8PFgIfAQUFMSw2MzBkZAIxDw8WAh8BBQUxLDkwNWRkAjMPDxYCHwEFBjE5LDI0M2RkAjUPDxYCHwEFBjI3LDY5OGRkAjcPDxYCHwEFATBkZAI5Dw8WAh8BBQEwZGQCOw8PFgIfAQUGNDcsNTQ5ZGQCPQ8PFgIfAQUGMTcsNzk2ZGQCPw8PFgIfAQUFMSw3MTFkZAJBDw8WAh8BBQUxLDAwMGRkAkMPDxYCHwEFAzQwMGRkAkUPDxYCHwEFAzQwMGRkAkcPDxYCHwEFCjM3LDk5OCwxOTVkZAJJDw8WAh8BBQkxLDUyMSw1OTNkZAJLDw8WAh8BBQEwZGQCTQ8PFgIfAQUBMGRkAk8PDxYCHwEFATBkZAIHD2QWUAIBDw8WAh8BBQkxMDUwMDAwNTZkZAIDDw8WAh8BBQkxMDUvMDYvMjFkZAIFDw8WAh8BBQkxMDUvMDkvMjFkZAIHDw8WAh8BBQo4NCw1ODAsMDUwZGQCCQ8PFgIfAQUKNDgsODcwLDEyNGRkAgsPDxYCHwEFAjM2ZGQCDQ8PFgIfAQUCMjdkZAIPDw8WAh8BBQIwMmRkAhEPDxYCHwEFAjQ5ZGQCEw8PFgIfAQUCMzVkZAIVDw8WAh8BBQIzNGRkAhcPDxYCHwEFAjI2ZGQCGQ8PFgIfAQUCMDJkZAIbDw8WAh8BBQIyN2RkAh0PDxYCHwEFAjM0ZGQCHw8PFgIfAQUCMzVkZAIhDw8WAh8BBQIzNmRkAiMPDxYCHwEFAjQ5ZGQCJQ8PFgIfAQUCMjZkZAInDw8WAh8BBQEwZGQCKQ8PFgIfAQUBMWRkAisPDxYCHwEFAjI4ZGQCLQ8PFgIfAQUCNjlkZAIvDw8WAh8BBQUxLDM2MmRkAjEPDxYCHwEFBTEsOTI3ZGQCMw8PFgIfAQUGMTksNzY3ZGQCNQ8PFgIfAQUGMjQsOTgyZGQCNw8PFgIfAQUBMGRkAjkPDxYCHwEFCTMsOTUyLDY0MmRkAjsPDxYCHwEFBjU3LDE5OGRkAj0PDxYCHwEFBjE1LDQ3M2RkAj8PDxYCHwEFBTIsMTU1ZGQCQQ8PFgIfAQUFMSwwMDBkZAJDDw8WAh8BBQM0MDBkZAJFDw8WAh8BBQM0MDBkZAJHDw8WAh8BBQoxOSw0ODUsNDcxZGQCSQ8PFgIfAQUBMGRkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAggPZBZQAgEPDxYCHwEFCTEwNTAwMDA1NWRkAgMPDxYCHwEFCTEwNS8wNi8xN2RkAgUPDxYCHwEFCTEwNS8wOS8xN2RkAgcPDxYCHwEFCzEzNiwyMjMsNTUwZGQCCQ8PFgIfAQULMjE2LDQ0MSw2OTFkZAILDw8WAh8BBQIxN2RkAg0PDxYCHwEFAjIxZGQCDw8PFgIfAQUCMzFkZAIRDw8WAh8BBQIxMGRkAhMPDxYCHwEFAjA3ZGQCFQ8PFgIfAQUCMjBkZAIXDw8WAh8BBQI0OWRkAhkPDxYCHwEFAjA3ZGQCGw8PFgIfAQUCMTBkZAIdDw8WAh8BBQIxN2RkAh8PDxYCHwEFAjIwZGQCIQ8PFgIfAQUCMjFkZAIjDw8WAh8BBQIzMWRkAiUPDxYCHwEFAjQ5ZGQCJw8PFgIfAQUBMWRkAikPDxYCHwEFATBkZAIrDw8WAh8BBQI1MWRkAi0PDxYCHwEFAzEwNmRkAi8PDxYCHwEFBTIsODIzZGQCMQ8PFgIfAQUFMywwNzhkZAIzDw8WAh8BBQYzMiw2NTlkZAI1Dw8WAh8BBQY0OCw5OTFkZAI3Dw8WAh8BBQsxNzAsMTIzLDc1M2RkAjkPDxYCHwEFATBkZAI7Dw8WAh8BBQY0NiwwOTlkZAI9Dw8WAh8BBQYxNCw3ODZkZAI/Dw8WAh8BBQUxLDUyNmRkAkEPDxYCHwEFBTEsMDAwZGQCQw8PFgIfAQUDNDAwZGQCRQ8PFgIfAQUDNDAwZGQCRw8PFgIfAQUBMGRkAkkPDxYCHwEFCTIsMzUxLDA5N2RkAksPDxYCHwEFATBkZAJNDw8WAh8BBQEwZGQCTw8PFgIfAQUBMGRkAgkPZBZQAgEPDxYCHwEFCTEwNTAwMDA1NGRkAgMPDxYCHwEFCTEwNS8wNi8xNGRkAgUPDxYCHwEFCTEwNS8wOS8xNGRkAgcPDxYCHwEFCzEyMSwyMDgsNDAwZGQCCQ8PFgIfAQULMTgxLDc4MSw3MjdkZAILDw8WAh8BBQIzM2RkAg0PDxYCHwEFAjEyZGQCDw8PFgIfAQUCMzdkZAIRDw8WAh8BBQIzNWRkAhMPDxYCHwEFAjEzZGQCFQ8PFgIfAQUCMTZkZAIXDw8WAh8BBQIxOGRkAhkPDxYCHwEFAjEyZGQCGw8PFgIfAQUCMTNkZAIdDw8WAh8BBQIxNmRkAh8PDxYCHwEFAjMzZGQCIQ8PFgIfAQUCMzVkZAIjDw8WAh8BBQIzN2RkAiUPDxYCHwEFAjE4ZGQCJw8PFgIfAQUBMGRkAikPDxYCHwEFATJkZAIrDw8WAh8BBQIzOWRkAi0PDxYCHwEFAjk4ZGQCLw8PFgIfAQUFMiwwODlkZAIxDw8WAh8BBQUyLDg1NGRkAjMPDxYCHwEFBjMwLDI3N2RkAjUPDxYCHwEFBjM4LDgzM2RkAjcPDxYCHwEFATBkZAI5Dw8WAh8BBQkxLDA4NCw5OThkZAI7Dw8WAh8BBQY1NSw2NDBkZAI9Dw8WAh8BBQYxNCw3NjFkZAI/Dw8WAh8BBQUxLDkwNGRkAkEPDxYCHwEFBTEsMDAwZGQCQw8PFgIfAQUDNDAwZGQCRQ8PFgIfAQUDNDAwZGQCRw8PFgIfAQULMTQxLDUxOCw3MzlkZAJJDw8WAh8BBQEwZGQCSw8PFgIfAQUBMGRkAk0PDxYCHwEFATBkZAJPDw8WAh8BBQEwZGQYAQUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFHUxvdHRvNjQ5Q29udHJvbF9oaXN0b3J5JHJhZE5PBR1Mb3R0bzY0OUNvbnRyb2xfaGlzdG9yeSRyYWROTwUdTG90dG82NDlDb250cm9sX2hpc3RvcnkkcmFkWU2y8kSumpbn387PI67yQ++NS4xwKh+5kpSNfthT4CR/pA=="
    event_validation = "/wEWHwLTmdzjBALPp5K3AwLfyLjZDwLAyLjZDwLByLjZDwLAyPTaDwLEyLjZDwLFyLjZDwLGyLjZDwLDyLjZDwLCyLjZDwLAyPjaDwKloeTwDgLeqs/fDgKjoZj6BQKozL21AgLH9t/DDwLimfnoBQLNnOeWBQLMnOeWBQLPnOeWBQLOnOeWBQLJnOeWBQLInOeWBQLLnOeWBQLanOeWBQLVnOeWBQLNnKeVBQLNnKuVBQLNnK+VBQLKut/DCKM6UIy4JyR7eXvvOn+2nZg9YiIlbe3cnDyYbb06ZuQM"
    url = "http://www.taiwanlottery.com.tw/lotto/Lotto649/history.aspx"
    
    body = {'__EVENTTARGET': '',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS':'',
        '__VIEWSTATE': view_state,
        '__VIEWSTATEGENERATOR':'C3E8EA98',
        '__EVENTVALIDATION': event_validation,
        'Lotto649Control_history$DropDownList1':'2',
        'Lotto649Control_history$chk':'radYM',
        'Lotto649Control_history$dropYear': year,
        'Lotto649Control_history$dropMonth': month,
        'Lotto649Control_history$btnSubmit':'查詢'}
    
    res = requests.post(url ,data = body)
    
    doc = html.fromstring(res.text)
    
    tmp_1 = list(filter(None, [c.text for c in doc.xpath("//td/span")]))
    tmp_1 = [x.replace(',', '') for x in tmp_1]
    data_1 = [sum([[tmp_1[i]], tmp_1[i+2:i+11], tmp_1[i+17:i+35]], []) for i in range(0, len(tmp_1), 35)]
    df_1 = pd.DataFrame(data_1)
    
    tmp_2 = [c.text for c in doc.xpath("//td/span/span")]
    tmp_2 = [x.replace(',', '') for x in tmp_2]
    data_2 = [sum([tmp_2[i:i+2], tmp_2[i+3:i+6]], []) for i in range(0, len(tmp_2), 6)]
    df_2 = pd.DataFrame(data_2)

    df = pd.concat([df_1, df_2], axis = 1)
    
    df.columns = ['期別', '兌獎截止', '銷售金額', '獎金總額', '獎號_1', '獎號_2', '獎號_3', '獎號_4', '獎號_5', '獎號_6', '貳獎_中獎注數', '參獎_中獎注數', '肆獎_中獎注數', '伍獎_中獎注數', '陸獎_中獎注數', '柒獎_中獎注數', '普獎_中獎注數', '貳獎_單注獎金', '參獎_單注獎金', '肆獎_單注獎金', '伍獎_單注獎金', '陸獎_單注獎金', '柒獎_單注獎金', '普獎_單注獎金', '貳獎_累積至次期獎金', '參獎_累積至次期獎金', '肆獎_累積至次期獎金', '伍獎_累積至次期獎金', '開獎日', '特別號', '頭獎_中獎注數', '頭獎_單注獎金', '頭獎_累積至次期獎金']
    
    new_column_order = ['期別', '開獎日', '兌獎截止', '銷售金額', '獎金總額', '獎號_1', '獎號_2', '獎號_3', '獎號_4', '獎號_5', '獎號_6', '特別號', '頭獎_中獎注數','貳獎_中獎注數', '參獎_中獎注數', '肆獎_中獎注數', '伍獎_中獎注數', '陸獎_中獎注數', '柒獎_中獎注數', '普獎_中獎注數', '頭獎_單注獎金','貳獎_單注獎金', '參獎_單注獎金', '肆獎_單注獎金', '伍獎_單注獎金', '陸獎_單注獎金', '柒獎_單注獎金', '普獎_單注獎金', '頭獎_累積至次期獎金','貳獎_累積至次期獎金', '參獎_累積至次期獎金', '肆獎_累積至次期獎金', '伍獎_累積至次期獎金']
    
    return df[new_column_order]

In [3]:
df = pd.DataFrame()
for year in (103, 104, 105):
    for month in range(1, 13, 1):
        sleep(0.5)
        try:
            df = pd.concat((getData(year, month), df), axis = 0)
        except (ValueError):
            break

In [6]:
df.to_csv("lottery_utf8.csv", index = False)